In [ ]:
#@title Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [25]:
#@title Step 2 : Run this cell
import sys
import shutil                                             #To play with directories and files
import os                                                 #To play with directories and files
import requests                                           #for getting Http request
from IPython.display import clear_output, display         #for clearing display
from subprocess import Popen, PIPE, getoutput             #to run bash commands
import re                                                 #regular expression for searching through html 
from bs4 import BeautifulSoup                             #to get html of the page (web scrapping)
import time                                               #for delaying exec
from google.colab import files                            #to play with gdrive files

#Installing Xattr for Gdrive file links
Popen(["apt-get","install","xattr",">","/dev/null"]).communicate()

#Updating settings.joson of transmission
fb = open("settings.json","w")
fb.write('''{
    "alt-speed-down": 50,
    "alt-speed-enabled": false,
    "alt-speed-time-begin": 540,
    "alt-speed-time-day": 127,
    "alt-speed-time-enabled": false,
    "alt-speed-time-end": 1020,
    "alt-speed-up": 50,
    "bind-address-ipv4": "0.0.0.0",
    "bind-address-ipv6": "::",
    "blocklist-enabled": false,
    "blocklist-url": "http://www.example.com/blocklist",
    "cache-size-mb": 4,
    "dht-enabled": true,
    "download-dir": "/var/lib/transmission-daemon/downloads",                   
    "download-limit": 100,
    "download-limit-enabled": 0,
    "download-queue-enabled": true,
    "download-queue-size": 5,
    "encryption": 1,
    "idle-seeding-limit": 30,
    "idle-seeding-limit-enabled": false,
    "incomplete-dir": "/var/lib/transmission-daemon/Downloads",
    "incomplete-dir-enabled": false,
    "lpd-enabled": false,
    "max-peers-global": 200,
    "message-level": 2,
    "peer-congestion-algorithm": "",
    "peer-id-ttl-hours": 6,
    "peer-limit-global": 200,
    "peer-limit-per-torrent": 50,
    "peer-port": 51413,
    "peer-port-random-high": 65535,
    "peer-port-random-low": 49152,
    "peer-port-random-on-start": false,
    "peer-socket-tos": "default",
    "pex-enabled": true,
    "port-forwarding-enabled": false,
    "preallocation": 1,
    "prefetch-enabled": true,
    "queue-stalled-enabled": true,
    "queue-stalled-minutes": 30,
    "ratio-limit": 2,
    "ratio-limit-enabled": false,
    "rename-partial-files": true,
    "rpc-authentication-required": true,
    "rpc-bind-address": "0.0.0.0",
    "rpc-enabled": true,
    "rpc-host-whitelist": "",
    "rpc-host-whitelist-enabled": true,
    "rpc-password": "{87954ec5cf3137559ab4ee22bccf7442a1f8f814MUAK.Gh9",
    "rpc-port": 9091,
    "rpc-url": "/transmission/",
    "rpc-username": "transmission",
    "rpc-whitelist": "127.0.0.*",
    "rpc-whitelist-enabled": true,
    "scrape-paused-torrents-enabled": true,
    "script-torrent-done-enabled": false,
    "script-torrent-done-filename": "",
    "seed-queue-enabled": false,
    "seed-queue-size": 10,
    "speed-limit-down": 100,
    "speed-limit-down-enabled": false,
    "speed-limit-up": 100,
    "speed-limit-up-enabled": false,
    "start-added-torrents": true,
    "trash-original-torrent-files": false,
    "umask": 18,
    "unmask": 2,
    "upload-limit": 100,
    "upload-limit-enabled": 0,
    "upload-slots-per-torrent": 14,
    "utp-enabled": true
}''')
fb.close()

#Running transmission installations
def Must_Run():

  #Check if Gdrive is mounted
  if(not(os.path.isdir("/content/drive"))):
    print("MOUNT YOUR GOOGLE DRIVE FIRST!!")
    sys.exit()
   
  #create Downloads folder if not present
  if(not(os.path.isdir("/content/drive/My Drive/Downloads"))):
    os.mkdir("/content/drive/My Drive/Downloads")


  #remove transmission if any (used to flush ID of previous torrent)
  try:
    Popen(["sudo","service","transmission-daemon","stop"]).communicate()
    Popen(["sudo","apt-get","remove","-y","transmission-cli","transmission-common","transmission-daemon"]).communicate()
  except:
    pass
  try:
    shutil.rmtree("/root/.config/transmission-daemon/")
  except:
    pass
  try:
    shutil.rmtree("/var/lib/transmission-daemon/") 
  except:
    pass

  #install transmission
  Popen(["sudo","add-apt-repository","-y","ppa:transmissionbt/ppa"]).communicate()
  Popen(["sudo","apt-get","install","transmission-cli","transmission-common","transmission-daemon"]).communicate()

  #updating settings of transmission
  Popen(["sudo","service","transmission-daemon","stop"]).communicate()
  os.remove("/var/lib/transmission-daemon/info/settings.json")
  Popen(["cp","settings.json","/var/lib/transmission-daemon/info/settings.json"]).communicate()
  Popen(["sudo","service","transmission-daemon","start"]).communicate()
  time.sleep(0.5)
  return




# Creating Torrent Class
class Torrent:

  def __init__(self):
    self.Title_list = []          #variable for Title list
    self.magnet_list = []         #variable for Magnet list
    self.detail_list = []         #variable for detail list (size, seeders, leechers)
    self.detail_append = []       
    self.date_list = []           #variable for upload date list
    self.download_list = []       #variable for download list
    self.page = 1                 #Variable to indicate current page
    self.search_term=''           #Variable for search term
    self.down_mem = 0             #Variable for total torrent memory


  #function to enter serach term
  def get_search(self):
    self.page = 1
    self.search_term = input('Enter Search Term  :  ')
    self.Search_list()
 
  #get free disk space of gdrive
  def get_free_space(self):
    total, used, free = shutil.disk_usage("/content/drive/")
    return(free // (2**20))

  #function to get search list
  def Search_list(self):
    self.Title_list = []
    self.magnet_list = []
    self.detail_list = []
    detail_append = []
    self.date_list = []
    det=0                     #temporary variable to get torrent details

    #Piratebay.party is the root website that is being scrapped with page number
    search_url = 'https://piratebay.party/search/' + self.search_term + '/'+str(self.page)+'/99/0'

    #getting search results from piratebay.party
    search_page = requests.get(search_url)

    #converting to html
    search_soup = BeautifulSoup(search_page.text, "lxml")

    #scrapping through HTML
    for link in search_soup.find_all("td"):

            #Getting titles of results
            if 'title="Details for'  in str(link):
                self.Title_list.append(re.search(r'title="Details for (.*?)">', str(link)).group(1))

            #Getting magnet links of results    
            if 'magnet:?'  in str(link):
                self.magnet_list.append(re.search(r'<a href="(.*?)" title', str(link)).group(1))

            #Getting torrent details    
            if 'align="right"'  in str(link):
                detail_append.append(re.search(r'"right">(.*?)</td>', str(link)).group(1))
                det+=1
            if (det==3):
                self.detail_list.append(detail_append)
                det=0
                detail_append=[]

            #Getting torrent upload dates
            if re.findall(r'\d\d-\d\d\s\d{4}', str(link)) or re.findall(r'\d\d:\d\d',str(link)):
                self.date_list.append(re.search(r'<td>(.*?)</td>', str(link)).group(1))
            
    #displaying the results
    if (self.Title_list==[]):
        
        #message if no results found
        print(f"Sorry! No Search Results")

    else:
        
        #printing if results found
        print(f"|Sl. No| {' ':<31}|TITLE|{' ':<30}    |Upload Date|     |Size|     |Seeders|  |Leachers| ")
        for index in range(0,len(self.Title_list)):
            k = index+1
            try:
               print(f"   {k:<8} {self.Title_list[index]:<70} {self.date_list[index]:<14} {self.detail_list[index][0]:<14} {self.detail_list[index][1]:<10} {self.detail_list[index][2]:<12}")
            except:
               pass
    return
 
  

  #function to add torrent to download list using search feature
  def add_download_Op1(self):

    #Getting index number of torrent to be added
    while (1):
      index_check=False
      index_list = input('Enter |Sl. No.| of Torrent to be added to Download ex:(1 or 1,2,3) or press 0 to exit  :  ').split(",")
      try:
        index_list=[int(item) for item in index_list]
      except:
        printf('Enter Valid Number')
        continue
      for item in index_list:
        if (len(self.Title_list)<item<=0):
          print('Enter Valid Number')
          index_check=True
          break
      if(index_check):
        continue
      break


    #Adding torrent to download list
    for index in index_list:  

       #Getting size of torrent to be downloaded
       if (self.detail_list[index-1][0][-3:] == 'MiB'):
         temp_size= float(self.detail_list[index-1][0][:-3])
       elif (self.detail_list[index-1][0][-3:]=='GiB'):
         temp_size= (float(self.detail_list[index-1][0][:-3]) * (2**10))
       elif (self.detail_list[index-1][0][-3:]=='KiB'):
         temp_size= (float(self.detail_list[index-1][0][:-3]) // (2**10))
       self.download_list.append([self.Title_list[index-1],temp_size,self.magnet_list[index-1]])
    return
     
  #function to get .torrent files
  def get_torrent_file(self):
      uploaded = files.upload()
      self.magnet_list = self.magnet_list + list(uploaded.keys())
      return  

  #function to get magnet links   
  def get_magnet_link(self):
      entry = input("Enter Magnet link/torrent Url : ")
      if(entry.lower()=='exit' or entry==''):
        pass
      else:
        self.magnet_list.append(entry)
      return 

  #function to Add torrent files and magnet links to download list   
  def add_download_Op2(self):

    tor_index=1

    #getting info of torrents by adding it
    for item in self.magnet_list:
        (stdout, stderr) = Popen(["transmission-remote","-n","transmission:transmission","-a",item], stdout=PIPE).communicate()  
        if("Error:" in stdout.decode("utf-8")):
          print(f"Torrent Entry {tor_index} : {item} \n Is Invalid!!")
          sys.exit()

    #getting name, size, magnet of torrents     
    for index in range(1,len(self.magnet_list)+1):
      while(1):
        (stdout, stderr) = Popen(["transmission-remote","-n","transmission:transmission","-t"+str(index),"-i"], stdout=PIPE).communicate()
        temp_list = stdout.decode("utf-8").split('\n')
        for i in range(15,18):
          if("Total size:" in temp_list[i]):
            temp_index=i
        if("None" in temp_list[temp_index]):
          continue
        else:
          temp_mem = temp_list[temp_index][14:].split(' ')
          if("MB" in temp_mem):
            temp_mem=float(temp_mem[0])
          elif("GB" in temp_mem):
            temp_mem=float(temp_mem[0])* (2**10)
          elif("KB" in temp_mem):
            temp_mem=float(temp_mem[0])// (2**10)
          self.download_list.append([temp_list[2][8:],temp_mem,temp_list[4][10:]])
          break
    
    return      

  #function to print download list
  def print_download_list(self):
     #get free disk space of gdrive
    rem_mem = self.get_free_space()

    #Displaying download list and checking if torrents can be downloaded into remaining gdrive memory   
    if (len(self.download_list)>0):
       print(f'Download List:\t\t\t Free Memory: {rem_mem} MiB')
       for index in range(0,len(self.download_list)):
          self.down_mem+=self.download_list[index][1]
          print(f'[{index+1}]    {self.download_list[index][0]:<70}     {str(self.download_list[index][1])+" MB":<14}')
       if (self.down_mem > rem_mem):
          print(f"WARNING: You Need extra {self.down_mem - rem_mem} MiB to Download correctly")
    return

  #function to remove torrent from download list
  def remove_download(self):

    #Getting index number of torrent to be removed
    while (1):
      index_check=False
      index_list = input('Enter |Sl. No.| of Torrent to be removed from Download ex:(1 or 1,2,3) or press 0 to exit  :  ').split(",")
      try:
        index_list=[int(item) for item in index_list]
      except:
        printf('Enter Valid Number')
        continue
      for item in index_list:
        if (len(self.Title_list)<item<=0):
          print('Enter Valid Number')
          index_check=True
          break
      if(index_check):
        continue
      break
    #Checking size of torrent to be downloaded after a torrent is removed
    for index in index_list:  
       self.download_list.pop(index-1)
       self.magnet_list.pop(index-1)   
    return
 
 
  #Downloading a torrent
  def Torrent_download(self):
    File_Name = []
    zip_var = False
    while (1):
        temp_var = input('Get zip compressed download link of torrent file? (Y/N):  ')
        if( temp_var.lower() =='y' ):
          zip_var = True
        elif(temp_var.lower() =='n'):
          pass
        else:
          printf('Enter Valid input')
          continue
        break
    
    #Adding torrent to download in transmission
    for item in self.download_list:
            Popen(["transmission-remote","-n","transmission:transmission","-a",item[2]]).communicate()
    
    #Getting index for downloads pending
    Down_pend = [num for num in range(1,len(self.download_list)+1)]

    #Getting index for downloads completed
    Down_Comp = []

    #list for adding download links
    print_list = []

    #flag to break from while loop
    break_flag = False

    while True:
      
      #clearing output
      clear_output(wait=True)

      #printing list output of transmission
      (stdout, stderr) = Popen(["transmission-remote","-n","transmission:transmission","-l"], stdout=PIPE).communicate()
      print(stdout.decode("utf-8"))
      for item in Down_Comp:
        print("\n"+item)
      if (zip_var):
        for printn in print_list:
          print("\n"+printn)

      #break from while if download completed and all download link generated 
      if (zip_var): 
        if (len(Down_pend)==0 and break_flag == True): 
            break
      else:
        if (len(Down_pend)==0): 
            break

      #Getting download links of files
      if (zip_var): 
        for tor_down in File_Name:
          fid = getoutput("xattr -p"+ " 'user.drive.id' "+"'/content/drive/My Drive/Downloads/"+tor_down+".zip"+"'")
          if (fid[0:5]=="local"):
            continue
          else:
            print_list.append("\n"+tor_down+".zip"+":\t"+"https://docs.google.com/uc?export/download&id="+fid)
            File_Name.remove(tor_down)
 
        if (len(print_list) == len(self.download_list)):
            break_flag = True
      
      #Getting Download completion status of torrent
      for count_1 in Down_pend:
        temp_var=stdout.decode("utf-8").split("\n")[count_1]
        if ("Done" in temp_var):
            tor_title = " ".join(temp_var.split()[9:])
            
            if(len(os.listdir("/var/lib/transmission-daemon/downloads/"+tor_title))!=0):
              File_Name.append(tor_title)
              if(zip_var):
                Popen(["zip","-r","/content/drive/MyDrive/Downloads/"+tor_title+".zip","/var/lib/transmission-daemon/downloads/"+tor_title]).communicate()
              else:
                Popen(["cp","/content/drive/MyDrive/Downloads/","/var/lib/transmission-daemon/downloads/"+tor_title]).communicate()
              Down_Comp.append(f"Torrent {count_1} Complete: {tor_title}")
              Down_pend.remove(count_1)
 
      time.sleep(1)
    return
 



# * Be Patient: torrenting takes more time depending on number of seerders
# * If torrent are stuck in idle or downloading stage please stop the cell and retry with another torrent



In [26]:
#@title Option 1: Search and Download Torrents (www.Piratebay.Party)
if __name__=="__main__":

  Must_Run()

#Create a torrent class 
  Test1 = Torrent()
  Test1.get_search()

  key=''
  while (True):
    #start searching torrent
    if (Test1.Title_list == []):
      Test1.get_search()
      continue
    
    #get input from user
    elif (len(Test1.download_list)==0):
      key = input("Enter 'D': to add Download, 'N': Next page, 'P': Previous page, 'S': change search term   :- ").strip()
      print(key)
      if (key.lower()=='d' or key.lower()=='n' or key.lower()=='p' or key.lower()=='s'):
        pass
      else:
        print("Enter Valid input!")
        continue
    else:
      key = input("Enter 'D': to add Download, 'R': Remove Download, 'N': Next page, 'P': Previous page, 'S': change search term, 'T':Start Downlaod  :- ")
      if (key.lower()=='d' or key.lower()=='n' or key.lower()=='p' or key.lower()=='s' or key.lower()=='t' or key.lower()=='r' ):
        pass
      else:
        print("Enter Valid input!")
        continue
     
    #Add to download list
    if (key.lower()=='d'):
      Test1.add_download_Op1()
      Test1.print_download_list()
  
    #Remove from download list
    if (key.lower()=='r'):
      Test1.remove_download()
      Test1.print_download_list()
    
    #Display next page
    if (key.lower()=='n'):
      clear_output(wait=True)
      time.sleep(1)
      Test1.page += 1
      Test1.Search_list()
    
    #Display previous page
    if (key.lower()=='p'):
      clear_output(wait=True)
      time.sleep(1)
      if (Test1.page==1):
        print("Previous Page Not Available!")
        continue
      Test1.page -= 1
      Test1.Search_list()

    #Search another term
    if (key.lower()=='s'):
      clear_output(wait=True)
      time.sleep(1)
      Test1.get_search()
    
    #start download
    if (key.lower()=='t'):
      Test1.Torrent_download()
      break

    ID   Done       Have  ETA           Up    Down  Ratio  Status       Name
     1   100%   111.0 MB  Done         0.0     0.0    0.0  Idle         Eminem - Kamikaze (2018) Mp3 (320kbps) [Hunter]
Sum:            111.0 MB               0.0     0.0


Torrent 1 Complete: Eminem - Kamikaze (2018) Mp3 (320kbps) [Hunter]


Eminem - Kamikaze (2018) Mp3 (320kbps) [Hunter].zip:	https://docs.google.com/uc?export/download&id=1-NAR1cE9Yp_dPjDOjVSscJT-bw00IIBg


In [ ]:
#@title Option 2 : Download Torrent using local torrent file or Magnet Link/Torrent url


if __name__=="__main__":

  Must_Run()

#Create a torrent class 
  Test2 = Torrent()

  key=''
  while (True):
    
    #get input from user
    if (len(Test2.magnet_list)==0):
      key = input("Enter 'A': to upload torrent file, 'M': to add Magnet Link :- ").strip()
      key = key.lower()
      if (key=='a' or key=='m'):
        pass
      else:
        print("Enter Valid input!")
        continue
    elif(len(Test2.magnet_list)!=0 and prev_key in ['a','m']):
      key = input("Enter 'A': to upload torrent file, 'M': to add Magnet Link, 'D': get Download list :- ")
      key = key.lower()
      if (key=='a' or key.lower()=='m' or key=='d' ):
        pass
      else:
        print("Enter Valid input!")
        continue
    elif(len(Test2.download_list)!=0 and prev_key in ['d','r']) :  
      key = input("Enter 'A': to upload torrent file, 'M': to add Magnet Link, 'R': Remove Download, 'T':Start Downlaod  :- ")
      key = key.lower()
      if (key=='a' or key=='m' or key=='r' or key=='t' ):
        pass
      else:
        print("Enter Valid input!")
        continue
    
    prev_key = key

    #Getting torrent inputs
    if (key.lower()=='a'):
      Test2.get_torrent_file()
    
    if (key.lower()=='m'):
      Test2.get_magnet_link()


    #Adding torrent to download list
    if (key.lower()=='d'):
      Test2.add_download_Op2()
      Test2.print_download_list()
      Must_Run()
  
    #Remove from download list
    if (key.lower()=='r'):
      Test2.remove_download()
      Test2.print_download_list()
      
    
    #start download
    if (key.lower()=='t'):
      Test2.Torrent_download()
      break